In [ ]:

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [4]:
def simple_chat(message, model="llama3.2"):
    """Simple chat completion with Ollama local model"""
    try:
        payload = {
            "model": model,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": message}
            ],
            "stream": False,
            "options": {
                "temperature": 0.7,
                "num_predict": 1000  # Ollama uses num_predict instead of max_tokens
            }
        }
        
        response = requests.post(OLLAMA_API, headers=HEADERS, json=payload)
        response.raise_for_status()  # Raises an exception for bad status codes
        
        result = response.json()
        return result["message"]["content"]
        
    except requests.exceptions.RequestException as e:
        return f"Network Error: {str(e)}"
    except KeyError as e:
        return f"Response format error: {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"
        

In [5]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [6]:
# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# summarize function for Ollama 
def summarize(url):
    website = Website(url)
    user_message = user_prompt_for(website)  # Using the user_prompt_for function
     # Using simple_chat function with Ollama
    response = simple_chat(user_message)
    return response

In [ ]:
# Test it
result = summarize("https://www.google.com")
display(Markdown(result))